In [32]:
import pandas as pd
import numpy as np
import os
import json

pd.set_option('display.max_columns', 500)

In [36]:
file='2.Indices_Target.csv'
df=pd.read_csv(f'C:/Users/511232/Desktop/Scorecard_coxcomb/{file}')
df.head(1)


,Goal_Num,dashboard_target,snapshot_count,proxy_count,mergid,CountryGroup,g_order,r_order,Goal,Target_number,Target_name_long,value,color,order,year,snapshot_target
0,1,0.0,1,0,1.1_Algeria,Algeria,6,1,1,1.1,Extreme poverty,0.0,1-Green,1,2015,0.0


In [54]:
'''
-read in the 2.Indices_Target.csv file
-filter on the NA-s in the snapshot_target column
-groupby countries and get the unique target numbers and write to a json file
'''
#filter out the dataframe where snapshot_target is NA
df_na=df[df['snapshot_target'].isna()].copy()

#groupby coutries adn get the all the unique targets per country and put it in a dictionary
country_target_NA_json=df_na.groupby(['CountryGroup']).apply(lambda x: set(x['Target_number'])).to_json()

#write to a josn file
with open('country_target.json', 'w') as outfile:
    outfile.write(country_target_NA_json)

In [52]:
'''
Code	Color
0	    Grey
1	    Red
2	    Yellow
3	    Green

-read in the json file and the scorecard.xlsx file
-for each of the country/target, filter color scheme=0 & Gap=blank
-loop over the countries and the unique target number from the json file, and average the "trend" column
-standardize the result of the average to be between [-10;+10]
formula=N= -10 + [(A+100)/10]

-generate coxcomb color using the following benchmarks:
if N is negative --> red, takes the value of 1
if N is postive but less or equal to 5.33 --> yellow, takes the value of 2
if N is greater than 5.33--> green, takes the value of 3"

'''

#read in the json file
with open('country_target.json', 'r') as outfile:
    j=json.load(outfile)